In [ ]:
#!/usr/bin/python3
# Author: Leo Cao
# Date: Jan 26, 2026
# Summary: This Python script uses TensorFlow to train a custom binary classifier and saves it for future use.

# Procedure:
# 0. Pet Images Data Set from GitHub: https://github.com/laxmimerit/dog-cat-full-dataset/archive/refs/heads/master.zip, extract the zip file, next, move folder data to the same path of the hol-ai-2601-cnn-cats-dogs, then rename it as PetImages and remove all non-jpg files. Download folder OwnImages from this repository to the same path.
# 1. Indentify cats and dogs
# 2. Predict from own images from 1.jpg to 9.jpg
# 3. Make prediction on multiple images
# 4. Save the Model as hol_ai_2601_cnn_cats_dogs_classifier_0226.keras

In [ ]:
# !nvidia-smi

In [ ]:
import tensorflow as tf

In [ ]:
# Check if a GPU is available 
# Explicitly place operations on the GPU (optional, often automatic) with tf.device('/device:GPU:0'):
if tf.test.gpu_device_name():
    print('Found GPU: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU found, using CPU")

In [ ]:
# Load the data, import keras, from keras.src.legacy.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1. / 255)
# Note: target_size needs to be a tuple (height, width)
training_set = train_datagen.flow_from_directory(
    './PetImages/train',
    target_size=(50, 50),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
# Show class indices
training_set.class_indices

In [ ]:
# Set up Convolutional Neural Networks (CNN) Model by Tensorflow Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

# Build Sequential instance
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(50,50,3)))

# Add Convolutional Layer with 32 filters, 3 colors (RGB), strides 1.
model.add(Conv2D(32, (3, 3), activation = 'relu'))
# Add Pool Layer
model.add(MaxPool2D(pool_size = (2,2)))

# Add Convolutional Layer with 32 filters, 3 colors (RGB), strides 1.
model.add(Conv2D(32, (3, 3), activation = 'relu'))
# Add Pool Layer
model.add(MaxPool2D(pool_size = (2,2)))

# Flattening Layer
model.add(Flatten())

# Fully Connection (FC) Layer
model.add(Dense(units = 128, activation = 'relu' ))
model.add(Dense(units = 1, activation = 'sigmoid' ))

In [ ]:
# Configure the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'] )
model.summary()

In [ ]:
# Then train the model with 25 epochs
model.fit(training_set, epochs = 25)

In [ ]:
# Print accuracy on the training data
accuracy_train = model.evaluate(training_set)
print(accuracy_train)

In [ ]:
# Print accuracy on the testing data
test_set = train_datagen.flow_from_directory(
    './PetImages/test',
    target_size=(50, 50),
    batch_size=32,
    class_mode='binary'
)
accuracy_test = model.evaluate(test_set)
print(accuracy_test)

In [ ]:
# Identify own jpg by loading single image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
# load cat and dog images
pic_cat = load_img('./OwnImages/cat.jpg', target_size=(50, 50))
pic_dog = load_img('./OwnImages/dog.jpg', target_size=(50, 50))

# Convert from image to array
pic_cat = img_to_array(pic_cat)
pic_dog = img_to_array(pic_dog)


# Normalization
pic_cat = pic_cat / 255
pic_dog = pic_dog / 255

# Convert to 3 dimension
pic_cat = pic_cat.reshape(1, 50, 50, 3)
pic_dog = pic_dog.reshape(1, 50, 50, 3)

result_cat = model.predict(pic_cat)
result_dog = model.predict(pic_dog)

print(result_cat)
print(result_dog)

In [ ]:
# Make prediction on multiple images
import matplotlib as mlp
font2 = {'family' : 'TeX Gyre Pagella',
'weight' : 'normal',
'size'   : 20,
}
mlp.rcParams['font.family'] = 'TeX Gyre Pagella'
mlp.rcParams['axes.unicode_minus'] = False

from matplotlib import pyplot as plt
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

a = [i for i in range(1,10)]
fig = plt.figure(figsize=(10,10))

for i in a:
    img_name = './OwnImages/'+str(i)+'.jpg'
    img_ori = load_img(img_name, target_size = (50, 50))
    img = img_to_array(img_ori)
    img = img.astype('float32') / 255
    img = img.reshape(1, 50, 50, 3)
    result = model.predict(img)
    img_ori = load_img(img_name, target_size = (250, 250))
    plt.subplot(3,3,i)
    plt.imshow(img_ori)
    plt.title (f'Dog: {result}' if result[0][0] == 1 else f'Cat: {result}')
plt.show()

In [ ]:
import tensorflow as tf

# Save the entire model as a `.keras` zip archive.
model.save('hol_ai_2601_cnn_cats_dogs_classifier_0226.keras')

print("Model saved successfully to hol_ai_2601_cnn_cats_dogs_classifier_0226.keras")

In [ ]:
import tensorflow as tf
# Reload a model from the .keras zip archive
model = tf.keras.models.load_model('hol_ai_2601_cnn_cats_dogs_classifier_0226.keras')

# Show the model architecture
model.summary()